# Using native TensorFlow ops to read images

```python
# Read an image from a file, decodes it into a dense tensor, and resize it to a fixed shape.


def _parse_function(filename, label):
  image_string = tf.read_file(filename)
  image_decoded = tf.image.decode_image(image_string)
  image_resized = tf.image.resize_images(image_decoded, [299, 299])

  return image_resized, label


# A vector of filenames
file_list = tf.gfile.Glob(filename)
filenames = tf.constant(file_list)

# labels[i] is the label for the image in filenames[i].

labels = tf.constant(label_list)
dataset = tf.data.Dataset.from_tensor_slices((filenames, labels))
dataset = dataset.map(_parse_function)

```

# Preprocess data into TFReocord

```python

def convert_to_example(csvline, categories):
  filename, label = csvline.encode('ascii', 'ignore').split(',')
  if label in categories:
    coder = ImageCoder()
    image_buffer, height, width = _get_image_data(filename, coder)
    example = _convert_to_example(filename, image_buffer, categories.index(label), label, height, width)

    yeild example.SerializeToString()

LABELS = ['nails', 'screws']

(p
  | beam.FlatMap(lambda line: convert_to_example(line, LABELS))
  | beam.io.tfrecordio.WriteToTFRecord(os.path.join(OUTPUT_DIR, 'train'))
)

```

```python

def input_fn(batch_size):
  files = tf.data.Dataset.list_files(file_pattern)
  dataset = tf.data.TFRecordDataset(files, num_parallel_reads=40)
  dataset = dataset.shuffle(buffer_size=10000)
  dataset = dataset.repeat(NUM_EPOCHS)
  dataset = dataset.map(preproc_fn, num_parallel_calls=40)
  dataset = dataset.batch(batch_size)

  # Prefetch pipelines everything above with the accelerator training
  dataset = dataset.prefetch(buffer_size=1)
  return dataset


```

# Using fused transformation ops

```python

def input_fn(batch_size):
  files = tf.data.Dataset.list_files(file_pattern)
  dataset = tf.data.TFRecordDataset(files, num_parallel_reads=40)


  dataset = dataset.apply(
    tf.contrib.data.shuffle_and_repeat(buffer_size=10000, NUM_EPOCHS)

  dataset = dataset.apply(
    tf.contrib.data.map_and_batch(parser_fn, batch_size))

  dataset = dataset.prefetch(buffer_size=1)

  return dataset


```

# Training with Estimator API

```python

# MirroredStrategy for multi GPU distribution
distribution = tf.contrib.distribute.MirroredStrategy()


# Pass the distribution to RunConfig
run_config = tf.estimator.RunConfig(train_distribute=distribution)

classifier = tf.estimator.Estimator(
  model_fn = model_function,
  model_dir = model_dir,
  config=run_config)

classifier.train(input_fn=input_function)

```

# `train_and_evaluate` bundles together a distributed workflow

```python

def train_and_evaluate(output_dir, config, params):
  features = [tf.feature_column.embedding_columns(...),
              tf.feature_column.bucketized_column(...)]
  
  estimator = tf.estimator.Estimator(model_fn = simple_rnn,
                                     model_dir = output_dir)

  train_spec = tf.estimator.TrainSpec(input_fn = get_train(),
                                      max_steps = 1000)

  exporter = tf.estimator.LatestExporter('exporter', serving_input_fn)
  eval_spec = tf.estimator.EvalSpec(input_fn = get_valid(),
                                    steps = None,
                                    exporters = exporter)
  tf.estimator.train_and_evaluate(estimator, train_spec, eval_spec)


# Runs training, evaluation, etc. on Cloud ML
train_and_evaluate(output_dir)



```

In [ ]:
``